In [1]:
import pandas as pd
hi=pd.DataFrame()
from bs4 import BeautifulSoup
from tensorflow import keras
import tensorflow as tf
from keras import layers
from utils import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
path='/data/fulltext_wiki_protected.csv'
all_articles = pd.read_csv(path, sep=',')[['title', 'bytes', 'full_text']]
i = 1
df = preprocess_classify_wiki_text(all_articles['full_text'].iloc[0])
while i < len(all_articles):
    working_df = preprocess_classify_wiki_text(all_articles['full_text'].iloc[i])
    df = pd.concat([df, working_df])
    i += 1
df.to_csv('/data/wiki_all_sentence.csv')


In [20]:
print(df)

                                                  text  label
0    Rudolf Walter Richard Heß  (* 2 April 1894 in ...   True
1    Heß war ab 1933 Reichsminister ohne Geschäftsb...  False
2    Öffentlich tat sich Heß als fanatischer Anhäng...  False
3    1933 ernannte ihn Adolf Hitler zu seinem Stell...  False
4    Am 1 Mai 1941 flog Heß in das Vereinigte König...  False
..                                                 ...    ...
683  Dieser Missionar macht geltend, daß durch die ...   True
684  Die Operation wird ja gerade zu einer Zeit aus...   True
685  Diese Art der intellektuellen Blindheit läßt s...   True
686  Dem Psychoanalytiker Matthias Franz zufolge, k...  False
687  Die Drohung heiße im Erleben vieler Jungen: „W...   True

[463745 rows x 2 columns]


In [ ]:
print(df_concateatd)

In [12]:
df=pd.read_csv('/data/wiki_all_sentence.csv')

In [14]:
df_true = df[df['label'] == True].sample(n=110677, random_state=2)
df_false = df[df['label'] == False].sample(n=110677, random_state=2)
df_concated = pd.concat([df_true, df_false])

In [18]:
print(df_concated)

        Unnamed: 0                                               text  label
172017          13  Er brach in Häuser und seine Schule ein, begin...   True
426014          36  Schlotterbeck köpfte gegen Borussia Mönchengla...   True
2681           116   Worum es mir und anderen reformistisch denken...   True
94154           81  Hätten diese es angenommen, wäre der Zweite We...   True
249295           9  Jede menschliche Niere verfügt über etwa eine ...   True
...            ...                                                ...    ...
31407          215    In dieser Region gab es 1836 immerhin 36 Zechen  False
156108         377  Mit den jüngsten Einwanderungswellen verstärkt...  False
145335         228  Die Trennung gelingt dabei nur früh im Jahr ge...  False
291282         379  von Mauretanien, ein Verbündeter Caesars im Kr...  False
348791         162  Es dauerte noch gut 150 Jahre, bis die letzten...  False

[221354 rows x 3 columns]


In [6]:
random_state=2
tf.random.set_seed(random_state)

model = keras.models.Sequential()

model.add(layers.Embedding(215, 32, input_length=32,embeddings_initializer=tf.keras.initializers.RandomUniform(seed=random_state)))
model.add(layers.LSTM(200, dropout=0.1,recurrent_initializer=tf.keras.initializers.Orthogonal(seed=42)))
model.add(layers.Dense(1, activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(seed=42)))
# model.add(layers.LSTM(wandb.config.hidden_layer_size, dropout=wandb.config.LSTM_dropout))
# model.add(layers.Dense(1, activation=wandb.config.dense_activation))
print(model.to_json())


{"class_name": "Sequential", "config": {"name": "sequential_1", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 32], "dtype": "float32", "sparse": false, "ragged": false, "name": "embedding_input"}}, {"class_name": "Embedding", "config": {"name": "embedding", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "input_dim": 215, "output_dim": 32, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": 2}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": 32}}, {"class_name": "LSTM", "config": {"name": "lstm", "trainable": true, "dtype": "float32", "return_sequences": false, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units": 200, "activation": "tanh", "recurrent_activation": "sigmoid", "use_bias": true, "kernel_initializer": {"class_name": "Gloro

In [63]:
import keras.losses
from keras.callbacks import EarlyStopping
from tensorflow import keras
from tensorflow.keras import callbacks, layers

loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.01)
# Define a wrapper function that returns the name and arguments of the loss function as a string.
def hash_fn_wrapper(loss_fn):
    hash_value = hash(str(loss_fn.__class__.__name__) + str(loss_fn.__dict__))
    print(str(loss_fn.__class__.__name__) + str(loss_fn.__dict__))
    return hash_value


hashable_loss = hash_fn_wrapper(optim)

print(hashable_loss)




Adam{'_use_locking': True, '_name': 'Adam', '_hyper': {'learning_rate': 0.01, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999}, '_slots': {}, '_slot_names': [], '_weights': [], '_iterations': None, '_deferred_slot_restorations': {}, '_initial_decay': 0.0, '_hypers_created': False, '_distribution_strategy': None, 'gradient_aggregator': <function all_reduce_sum_gradients at 0x2930f4430>, 'gradient_transformers': [], '_clipnorm': None, '_clipnorm_fn': <function make_gradient_clipnorm_fn.<locals>.<lambda> at 0x293725ee0>, '_global_clipnorm': None, '_global_clipnorm_fn': <function make_global_gradient_clipnorm_fn.<locals>.<lambda> at 0x2937258b0>, '_clipvalue': None, '_clipvalue_fn': <function make_gradient_clipvalue_fn.<locals>.<lambda> at 0x293c45700>, 'epsilon': 1e-07, 'amsgrad': False}
4682916816146857785


In [53]:
import keras
from keras import losses
import hashlib
def loss_wrapper(loss_fn):
    return loss_fn.__name__

loss = keras.losses.BinaryCrossentropy(from_logits=False)
print(loss_wrapper(loss))
print(str(loss))
# loss_str = loss
# inter_model_hash = hashlib.sha256(loss_str.encode()).hexdigest()

AttributeError: 'BinaryCrossentropy' object has no attribute '__name__'

In [70]:
def fetch_wiki_fulltext_linklabeled(subject='Maschinelles Lernen'):
    """Fetches Raw Text from specified Wiki article

    Args:
        subject (str, optional): Name of Wiki-Article. Defaults to 'Maschinelles Lernen'.

    Returns:
        str: Raw text from wiki article (including citations)
    """
    parse_url = 'https://de.wikipedia.org/w/api.php'
    params_parse = {'action': 'parse', 'page': subject, 'format': 'json', 'prop': 'text|links|linkshere', 'redirects': ''}

    # Make the API request
    response_parse = requests.get(parse_url, params=params_parse).json()
    raw_html = response_parse['parse']['text']['*']
    soup = BeautifulSoup(raw_html, 'html.parser')
    soup.find_all('p')
    text = ''
    for p in soup.find_all('p'):
        text += p.text

    processed_text=''
    p_elements = soup.find_all('p')
    print(p_elements)
    for p in p_elements:
        # Iterate over each child element of the p element
        for child in p.children:
            if child.name == None:
                # If the child element is a string, add it to the processed text
                processed_text += child
            elif child.name == "a":
                # If the child element is an "a" tag, add a "@" after it
                processed_text += child.text + " @@ "
            elif child.name == "b":
                processed_text += child.text
            else:
                # Ignore all other child elements
                pass
        # Add a newline character after each p element
        processed_text += "\n"
    return processed_text
a = fetch_wiki_fulltext_linklabeled('Steve Jobs')
# import requests

# # Set the Wikipedia API URL and parameters
# wiki_api_url = "https://de.wikipedia.org/w/api.php"
# params = {
#     "action": "query",
#     "format": "json",
#     "prop": "links|iws",
#     "titles": "Maschinelles Lernen",
#     "pllimit": "max",
#     "iwlimit": "max",
#     "iwprefix": "en"
# }

# # Make the API request
# response = requests.get(wiki_api_url, params=params).json()

# # Get the page ID of the article
# page_id = next(iter(response["query"]["pages"]))

# # Get the links and interwiki links from the page data
# links = response["query"]["pages"][page_id]["links"]


# # Get a dictionary of link titles and anchor texts
# link_info = {}
# for link in links:
#     link_info[link["title"]] = link["title"]

# # Get the raw text of the article



# print(link_info)


[<p><b>Steven „Steve“ Paul Jobs</b> (* <a href="/wiki/24._Februar" title="24. Februar">24. Februar</a> <a href="/wiki/1955" title="1955">1955</a> in <a href="/wiki/San_Francisco" title="San Francisco">San Francisco</a>, <a href="/wiki/Kalifornien" title="Kalifornien">Kalifornien</a>;<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> † <a href="/wiki/5._Oktober" title="5. Oktober">5. Oktober</a> <a href="/wiki/2011" title="2011">2011</a> in <a href="/wiki/Palo_Alto" title="Palo Alto">Palo Alto</a>, Kalifornien<sup class="reference" id="cite_ref-Meldung_Steve_Jobs_ist_tot_2-0"><a href="#cite_note-Meldung_Steve_Jobs_ist_tot-2">[2]</a></sup><sup class="reference" id="cite_ref-apple-tod_3-0"><a href="#cite_note-apple-tod-3">[3]</a></sup>) war ein <a href="/wiki/Vereinigte_Staaten" title="Vereinigte Staaten">US-amerikanischer</a> <a href="/wiki/Unternehmer" title="Unternehmer">Unternehmer</a>. Als Mitgründer und langjähriger <a href="/wiki/Chief_Executive_Officer" ti

In [35]:
import requests

# Set the Wikipedia API URL and parameters
wiki_api_url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "prop": "links",
    "titles": "Machine learning",
    "pllimit": "max"
}

# Make the API request
response = requests.get(wiki_api_url, params=params).json()

# Get the page ID of the article
page_id = next(iter(response["query"]["pages"]))

# Get the links from the page data
links = response["query"]["pages"][page_id]["links"]

# Get a dictionary of link titles and anchor texts
link_info = {}
for link in links:
    link_info[link["title"]] = link["title"]

# Get the raw text of the article
print(response["query"]["pages"][page_id])

# Split the text into words
words = text.split()

# Add "@page" after words that contain a link
processed_text = ""
for word in words:
    if "[" in word and "]" in word and "|" in word:
        # Get the link title and anchor text
        link_title = word.split("|")[0][1:]
        anchor_text = word.split("|")[1][:-1]
        # Add the anchor text to the link_info dictionary
        link_info[link_title] = anchor_text
        # Add "@page" after the anchor text
        processed_text += f"{anchor_text}@page "
    elif word in link_info:
        # Add "@page" after the word
        processed_text += f"{word}@page "
    else:
        processed_text += f"{word} "

# Print the processed text
print(processed_text)


{'pageid': 233488, 'ns': 0, 'title': 'Machine learning', 'links': [{'ns': 0, 'title': '15.ai'}, {'ns': 0, 'title': 'AAAI Conference on Artificial Intelligence'}, {'ns': 0, 'title': 'ACM Computing Classification System'}, {'ns': 0, 'title': 'ACM Computing Surveys'}, {'ns': 0, 'title': 'ADALINE'}, {'ns': 0, 'title': 'AI control problem'}, {'ns': 0, 'title': 'AI takeover'}, {'ns': 0, 'title': 'AI winter'}, {'ns': 0, 'title': 'AT&T Labs'}, {'ns': 0, 'title': 'Action selection'}, {'ns': 0, 'title': 'Activation function'}, {'ns': 0, 'title': 'Active learning (machine learning)'}, {'ns': 0, 'title': 'Adaptive website'}, {'ns': 0, 'title': 'Adversarial machine learning'}, {'ns': 0, 'title': 'Affective computing'}, {'ns': 0, 'title': 'Agriculture'}, {'ns': 0, 'title': 'Alan Mackworth'}, {'ns': 0, 'title': 'Alan Turing'}, {'ns': 0, 'title': 'AlexNet'}, {'ns': 0, 'title': 'Alex Graves (computer scientist)'}, {'ns': 0, 'title': 'Algorithm'}, {'ns': 0, 'title': 'Algorithm design'}, {'ns': 0, 'title

In [36]:
import requests
from bs4 import BeautifulSoup

# Set the URL of the article
url = "https://en.wikipedia.org/wiki/Machine_learning"

# Make a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object from the response text
soup = BeautifulSoup(response.text, "html.parser")

# Find the text of the article
text = soup.find("div", class_="mw-parser-output").get_text()

# Split the text into words
words = text.split()

# Process each word
processed_words = []
for word in words:
    if "[" in word and "]" in word and "|" in word:
        # Get the link text and add "@page" after it
        link_text = word.split("|")[1][:-1]
        processed_words.append(link_text + "@page")
    elif "[" in word and "]" in word:
        # Replace citations with an underscore
        processed_words.append("_")
    else:
        processed_words.append(word)

# Join the processed words back into a string
processed_text = " ".join(processed_words)

# Print the processed text
print(processed_text)


Study of algorithms that improve automatically through experience For the journal, see Machine Learning (journal). "Statistical learning" redirects here. For statistical learning in linguistics, see statistical learning in language acquisition. Part of a series onMachine learningand data mining Paradigms Supervised learning Unsupervised learning Online learning Batch learning Semi-supervised learning Self-supervised learning Reinforcement learning Problems Classification Regression Clustering dimension reduction density estimation Anomaly detection Data Cleaning AutoML Association rules Structured prediction Feature engineering Feature learning Learning to rank Grammar induction Supervised learning(classification • regression) Decision trees Ensembles Bagging Boosting Random forest k-NN Linear regression Naive Bayes Artificial neural networks Logistic regression Perceptron Relevance vector machine (RVM) Support vector machine (SVM) Clustering BIRCH CURE Hierarchical k-means Fuzzy Expec

In [13]:
import requests
import re

# Set the Wikipedia API URL and parameters
wiki_api_url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "prop": "extracts|links",
    "titles": "Machine learning",
    "explaintext": "1",
    "pllimit": "max"
}

# Make the API request
response = requests.get(wiki_api_url, params=params).json()

# Get the page ID of the article
page_id = next(iter(response["query"]["pages"]))

# Get the raw text of the article
text = response["query"]["pages"][page_id]["extract"]

# Get the links from the page data
links = response["query"]["pages"][page_id]["links"]

# Get a list of link titles
link_titles = [link["title"] for link in links]

# Split the text into words
words = re.findall(r'\w+', text)

# Add "@page" after words that contain a link
processed_text = ""
for word in words:
    if word in link_titles:
        processed_text += f"{word}@page "
    else:
        processed_text += f"{word} "

# Print the processed text
print(processed_text)


Machine learning ML is a field of inquiry devoted to understanding and building methods that learn that is methods that leverage data to improve performance on some set of tasks It is seen as a part of artificial intelligence Machine learning algorithms build a model based on sample data known as training data in order to make predictions or decisions without being explicitly programmed to do so Machine learning algorithms are used in a wide variety of applications such as in medicine email filtering speech recognition agriculture and computer vision where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks A subset of machine learning is closely related to computational statistics which focuses on making predictions using computers but not all machine learning is statistical learning The study of mathematical optimization delivers methods theory and application domains to the field of machine learning Data mining is a related field of study foc

In [5]:
import requests

# Set the Wikipedia API URL and parameters
wiki_api_url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "prop": "links",
    "titles": "Steve jobs",
}

# Make the API request
response = requests.get(wiki_api_url, params=params).json()
print(response)
# Get the page ID of the article
page_id = next(iter(response["query"]["pages"]))

# Get the links from the page data
links = response["query"]["pages"][page_id]["links"]

# Print the links and their titles
for link in links:
    print(link)

{'continue': {'plcontinue': '1563047|14|Redirects_from_incorrect_names', 'continue': '||'}, 'query': {'pages': {'1563047': {'pageid': 1563047, 'ns': 0, 'title': 'Steve jobs', 'links': [{'ns': 0, 'title': 'H:L'}, {'ns': 0, 'title': 'H:S'}, {'ns': 0, 'title': 'Steve Jobs'}, {'ns': 4, 'title': 'Wikipedia:1'}, {'ns': 4, 'title': 'Wikipedia:Categorizing redirects'}, {'ns': 4, 'title': 'Wikipedia:Piped link'}, {'ns': 4, 'title': 'Wikipedia:Printability'}, {'ns': 4, 'title': 'Wikipedia:Protection policy'}, {'ns': 4, 'title': 'Wikipedia:Redirect'}, {'ns': 10, 'title': 'Template:R from incorrect name'}]}}}}
{'ns': 0, 'title': 'H:L'}
{'ns': 0, 'title': 'H:S'}
{'ns': 0, 'title': 'Steve Jobs'}
{'ns': 4, 'title': 'Wikipedia:1'}
{'ns': 4, 'title': 'Wikipedia:Categorizing redirects'}
{'ns': 4, 'title': 'Wikipedia:Piped link'}
{'ns': 4, 'title': 'Wikipedia:Printability'}
{'ns': 4, 'title': 'Wikipedia:Protection policy'}
{'ns': 4, 'title': 'Wikipedia:Redirect'}
{'ns': 10, 'title': 'Template:R from inco

NameError: name 'DICjson' is not defined

In [1]:
import tensorflow as tf 

In [3]:
print('Tensorflow version: @' + str(tf.__version__))

Tensorflow version: @2.11.0


In [2]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

Metal device set to: Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-13 16:26:54.540452: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-13 16:26:54.540491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


AttributeError: module 'tensorflow' has no attribute 'Session'